In [ ]:
"""
This notebook is used to extract the <token, id, canonical, variants> triple from the ado.owl

"""

In [ ]:
from bs4 import BeautifulSoup
from xml.dom import minidom
import xmltodict
from collections import Counter, OrderedDict
from collections.abc import Mapping

In [1]:
ado_path = "/Users/yidesdo21/Projects/inputs/ontologies/Alzheimer Ontology v15R-xml_merged.owl"

with open(ado_path) as f:
    contents = f.read()
#     print(contents)

soup = BeautifulSoup(contents, 'html.parser')

In [18]:
# create file
xmldoc = minidom.Document()

# creat root element
root_element = xmldoc.createElement('synonym')
xmldoc.appendChild(root_element)


for owl_class in soup.find_all("owl:class"):
    if not owl_class.findParent('owl:equivalentclass') and not owl_class.findParent('rdfs:subclassof'):
        rdf_about = owl_class["rdf:about"].replace("&", "")
        ontology = rdf_about.split(";")[0]
        token_id = rdf_about.replace(";", ":")
        
        productChild = xmldoc.createElement('token')
        productChild.setAttribute('id', token_id)  # attribute, value

        for child in owl_class.contents:
            variants = set()
            if child.name == "rdfs:label": 
                cano = child.get_text()
#                 cano = child.get_text().lower()

                if cano not in variants:   # avoid duplications in the variant base 
                    variants.add(cano)

                    productChild.setAttribute('canonical', cano)
                    root_element.appendChild(productChild)

                    # create child element, the canonical also has to be in the variant
                    product_grandChild = xmldoc.createElement('variant')

                    # insert user data into element
                    product_grandChild.setAttribute('base', cano)
                    productChild.appendChild(product_grandChild) 
                
#                 id_cano.append((token_id, cano))
                
            if child.name == "NDDUO:Synonym".lower():   # because beautifulsoup lowercase the children names
                syno = child.get_text()
#                 syno = child.get_text().lower()
#                 print(syno)
                
                if syno not in variants:  # avoid duplications in the variant base
                    variants.add(syno)
                    
                    # create child element
                    product_grandChild = xmldoc.createElement('variant')

                    # insert user data into element
                    product_grandChild.setAttribute('base', syno)
                    productChild.appendChild(product_grandChild)                  
                
#                 id_syno.append((token_id, syno))
        
xml_str = xmldoc.toprettyxml(indent ="\t")

# save file
save_path_file = "ado_case.xml"
dict_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(dict_path+save_path_file, "w") as f:
    f.write(xml_str)        
